In [36]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
import time
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import datetime
import pickle
import json
import collections
import itertools

In [24]:
df = pd.read_excel("totalstats21-22.xlsx")
df

,Unnamed: 0,Player,Tm,MP,FG,FGA,FG%,3P,3PA,3P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,BPM,OFFRTG,DEFRTG,+/-
0,0,Precious Achiuwa,TOR,23.6,3.6,8.3,0.439,0.8,2.1,0.359,0.486,1.1,1.8,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1,0.503,0.259,0.217,8.7,21.7,14.9,6.9,1.1,2.3,11.3,18.5,-2.6,108.7,106.7,72
1,1,Steven Adams,MEM,26.3,2.8,5.1,0.547,0.0,0.0,0.000,0.547,1.4,2.6,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9,0.560,0.003,0.518,17.9,22.0,19.9,16.1,1.6,2.7,19.6,12.0,2.0,117.8,109.5,353
2,2,Bam Adebayo,MIA,32.6,7.3,13.0,0.557,0.0,0.1,0.000,0.557,4.6,6.1,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1,0.608,0.008,0.466,8.7,26.1,17.5,17.5,2.2,2.6,14.4,25.0,3.8,113.1,104.9,300
3,3,Santi Aldama,MEM,11.3,1.7,4.1,0.402,0.2,1.5,0.125,0.424,0.6,1.0,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1,0.452,0.364,0.242,9.4,16.1,12.6,7.7,0.8,2.5,9.9,18.4,-5.7,105.9,104.6,16
4,4,LaMarcus Aldridge,BRK,22.3,5.4,9.7,0.550,0.3,1.0,0.304,0.566,1.9,2.2,0.873,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9,0.604,0.100,0.223,7.8,18.7,13.4,6.3,0.6,4.0,8.0,22.4,0.7,111.3,111.1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,757,Thaddeus Young,TOR,18.3,2.6,5.5,0.465,0.7,1.7,0.395,0.524,0.5,1.0,0.481,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,0.526,0.299,0.188,8.7,18.1,13.1,13.6,3.3,2.3,12.4,16.0,2.0,106.1,106.3,20
758,758,Trae Young,ATL,34.9,9.4,20.3,0.460,3.1,8.0,0.382,0.536,6.6,7.3,0.904,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,0.603,0.395,0.358,2.1,9.6,5.9,46.7,1.3,0.2,14.5,34.4,5.2,117.2,114.9,159
759,759,Omer Yurtseven,MIA,12.6,2.3,4.4,0.526,0.0,0.2,0.091,0.528,0.7,1.1,0.623,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,0.546,0.045,0.247,13.9,33.0,23.6,11.0,1.2,3.0,13.0,19.9,-1.0,113.3,108.0,63
760,760,Cody Zeller,POR,13.1,1.9,3.3,0.567,0.0,0.1,0.000,0.567,1.4,1.8,0.776,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,0.627,0.044,0.544,14.9,24.1,19.3,9.4,1.1,1.7,14.6,15.9,-2.1,108.8,110.9,-28


In [25]:
team_list = list(df.Tm.unique())
team_list = ['BRK' if item == 'BKN' else item for item in team_list]
team_list = ['CHO' if item == 'CHA' else item for item in team_list]
team_list = ['PHO' if item == 'PHX' else item for item in team_list]
team_list

['TOR',
 'MEM',
 'MIA',
 'BRK',
 'TOT',
 'NOP',
 'UTA',
 'MIL',
 'CLE',
 'IND',
 'LAL',
 'ORL',
 'NYK',
 'HOU',
 'WAS',
 'PHO',
 'SAC',
 'DET',
 'CHO',
 'CHI',
 'ATL',
 'DEN',
 'PHI',
 'SAS',
 'LAC',
 'OKC',
 'MIN',
 'GSW',
 'POR',
 'DAL',
 'BOS']

In [19]:
number = 1
driver = webdriver.Firefox()

power_rankings = {}

while(number <= 25):
    '''
    if(number == 2):
        url = 'https://www.nba.com/news/power-rankings-week-2-2'
    elif(number == 4 or number == 5):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-la-clippers"
    elif(number == 6 or number == 8):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-golden-state-warriors-cleveland-cavaliers"
    elif(number == 7):
        url = "https://www.nba.com/news/power-rankings-week-7-golden-state-warriors-san-antonio-spurs"
    elif(number == 9):
        url = "https://www.nba.com/news/power-rankings-week-9-houston-rockets-utah-jazz"
    elif(number == 10 or number == 11):
        url = f"https://www.nba.com/news/power-rankings-week-{number}-cleveland-cavaliers-golden-state-warriors"
    elif(number == 12):
        url = "https://www.nba.com/news/power-rankings-week-12-san-antonio-spurs-houston-rockets"
    else: 
    '''
    url = f"https://www.nba.com/news/power-rankings-2023-24-week-{number}"
    driver.get(url)
    src = driver.page_source
    parser = BeautifulSoup(src, "html.parser")
    ranks = parser.find_all("div", attrs = {"class": "ArticlePowerRankings_prTop__UrGiu"})
    published = parser.find('time', attrs = {"class": "ArticleHeader_ahDate__J3fwr"})
    time = published.contents[0]
    a_list = []
    for rank in ranks:
        a_tag = rank.find('a', href=True) 
        a_list.append(a_tag['href'])
    if('Updated on' in published.contents[0]):
        published.contents[0] = published.contents[0].replace('Updated on ', '')
    form = "%B %d, %Y %I:%M %p"
    new_date = datetime.datetime.strptime(published.contents[0], form).date()
    output_form = '%m-%d-%Y'
    processed_date = datetime.datetime.strftime(new_date, output_form)
    
    a_list = ['BRK' if item == '/nets' else item for item in a_list]
    a_list = ['CHO' if item == '/hornets/' else item for item in a_list]
    a_list = ['PHO' if item == '/suns/' else item for item in a_list]
    a_list = ['BOS' if item == '/celtics/' else item for item in a_list]
    a_list = ['MIL' if item == '/bucks/' else item for item in a_list]
    a_list = ['DEN' if item == '/nuggets/' else item for item in a_list]
    a_list = ['PHI' if item == '/sixers/' else item for item in a_list]
    a_list = ['CLE' if item == '/cavaliers' else item for item in a_list]
    a_list = ['MEM' if item == '/grizzlies/' else item for item in a_list]
    a_list = ['SAC' if item == '/kings' else item for item in a_list]
    a_list = ['NYK' if item == '/knicks/' else item for item in a_list]
    a_list = ['GSW' if item == '/warriors' else item for item in a_list]
    a_list = ['LAC' if item == '/clippers/' else item for item in a_list]
    a_list = ['LAL' if item == 'http://nba.com/lakers' else item for item in a_list]
    a_list = ['MIA' if item == '/heat/' else item for item in a_list]
    a_list = ['NOP' if item == '/pelicans' else item for item in a_list]
    a_list = ['OKC' if item == '/thunder' else item for item in a_list]
    a_list = ['MIN' if item == '/timberwolves' else item for item in a_list]
    a_list = ['CHI' if item == '/bulls' else item for item in a_list]
    a_list = ['TOR' if item == '/raptors/' else item for item in a_list]
    a_list = ['ATL' if item == '/hawks/' else item for item in a_list]
    a_list = ['DAL' if item == 'https://www.mavs.com' else item for item in a_list]
    a_list = ['UTA' if item == '/jazz/' else item for item in a_list]
    a_list = ['ORL' if item == '/magic' else item for item in a_list]
    a_list = ['WAS' if item == '/wizards' else item for item in a_list]
    a_list = ['IND' if item == '/pacers/' else item for item in a_list]
    a_list = ['POR' if item == '/blazers/' else item for item in a_list]
    a_list = ['HOU' if item == '/rockets' else item for item in a_list]
    a_list = ['SAS' if item == '/spurs/' else item for item in a_list]
    a_list = ['DET' if item == '/pistons' else item for item in a_list]
    
    power_rankings[processed_date] = a_list
    print('week ' + str(number) + ' done')
    
    number += 1
    
    
driver.close()
    

week 1 done
week 2 done
week 3 done
week 4 done
week 5 done
week 6 done
week 7 done
week 8 done
week 9 done
week 10 done
week 11 done
week 12 done
week 13 done
week 14 done
week 15 done
week 16 done
week 17 done
week 18 done
week 19 done
week 20 done
week 21 done
week 22 done
week 23 done
week 24 done
week 25 done


In [20]:
with open("powerrankings23-24.json", "w") as f:
    json.dump(power_rankings, f)


In [40]:
options = Options()
driver = webdriver.Chrome(service=Service(), options = options)
driver.get("https://www.basketball-reference.com/boxscores/202310240DEN.html")
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')
basic_tables = soup.find_all('table', id=re.compile(r'box-.*-game-basic'))
adv_tables = soup.find_all('table', id=re.compile(r'box-.*-game-advanced'))

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=47221): Read timed out. (read timeout=120)

In [38]:
#driver = webdriver.Firefox()
options = Options()
driver = webdriver.Chrome(service=Service(), options = options)
start_date = datetime.date(2023, 10, 24)
end_date = datetime.date(2024, 6, 17)
delta = datetime.timedelta(days = 1)

date_dict = {}

while(start_date < end_date):
    day2 = str(start_date.day)
    month2 = str(start_date.month)
    year2 = str(start_date.year)
    if(start_date.day < 10):
        day2 = "0" + day2
    if(start_date.month < 10):
        month2 = "0" + month2

    date = '' + month2 + '-' + day2 + '-' + year2  
    date_dict[date] = {}

    for team in team_list:
        url = f"https://www.basketball-reference.com/boxscores/{year2}{month2}{day2}0{team}.html"
        print(url)
        if response.status_code == 429:
            print(int(response.headers["Retry-After"]))
            time.sleep(int(response.headers["Retry-After"]))
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            continue

        driver.get(url)
        print("hi")
        time.sleep(8)
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        basic_tables = soup.find_all('table', id=re.compile(r'box-.*-game-basic'))
        adv_tables = soup.find_all('table', id=re.compile(r'box-.*-game-advanced'))

        #tables = soup.find_all('table', {'id': re.compile('box-.*-game-basic')})
        #advanced = soup.find_all('table', {'id': re.compile('box-.*-game-advanced')})

        print("found link")
        
        
        away = tables[0].get('id')[4:7]

        
        dfs = pd.read_html(str(tables), header=1)
        advanced_dfs = pd.read_html(str(advanced), header = 1)

        df_away = dfs[0]
        df_home = dfs[1]
        df_away_advanced = advanced_dfs[0]
        df_home_advanced = advanced_dfs[1]

        print(df_away)
        df_away = pd.merge(df_away, df_away_advanced, on = ['Starters', 'MP'])
        df_home = pd.merge(df_home, df_home_advanced, on = ['Starters', 'MP'])
        
        df_away['Home'] = 0
        df_home['Home'] = 1

        date_dict[date][team] = {}
        date_dict[date][away] = {}

        date_dict[date][team][away] = df_home
        date_dict[date][away][team] = df_away

    
    print("day done")
    start_date += delta


driver.close()

https://www.basketball-reference.com/boxscores/202310240TOR.html
https://www.basketball-reference.com/boxscores/202310240MEM.html
https://www.basketball-reference.com/boxscores/202310240MIA.html
https://www.basketball-reference.com/boxscores/202310240BRK.html
https://www.basketball-reference.com/boxscores/202310240TOT.html
https://www.basketball-reference.com/boxscores/202310240NOP.html
https://www.basketball-reference.com/boxscores/202310240UTA.html
https://www.basketball-reference.com/boxscores/202310240MIL.html
https://www.basketball-reference.com/boxscores/202310240CLE.html
https://www.basketball-reference.com/boxscores/202310240IND.html
https://www.basketball-reference.com/boxscores/202310240LAL.html
https://www.basketball-reference.com/boxscores/202310240ORL.html
https://www.basketball-reference.com/boxscores/202310240NYK.html
https://www.basketball-reference.com/boxscores/202310240HOU.html
https://www.basketball-reference.com/boxscores/202310240WAS.html
https://www.basketball-re

In [32]:
date_dict

{'10-24-2023': {}, '10-25-2023': {}, '10-26-2023': {}, '10-27-2023': {}}

In [12]:
with open('dict23-24.pkl', 'wb') as f:  # open a text file
    pickle.dump(date_dict, f)
    f.close()

In [ ]:
with open('boxREG.pkl', 'wb') as f:  # open a text file
    pickle.dump(teams_dict, f)
    f.close()

In [ ]:
#seasons = ['2012-13', '2013-14', '2014-15', '2015-16','2016-17','2017-18' ,'2018-19' ,'2019-20','2021-22', '2022-23']

In [ ]:
#for season in seasons:
    #print(f"my name is {season}")

In [ ]:
#timeout = 25

In [ ]:
#driver = webdriver.Firefox()

In [ ]:
'''def get_data(seasons):
    for season in seasons:
        url = f"https://www.nba.com/stats/teams/misc?Season={season}&SeasonType=Regular+Season"
        driver.get(url)
        #element_present = EC.presence_of_element_located((By.XPATH, r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
        #WebDriverWait(driver, timeout).until(element_present)
        #select = Select(driver.find_element("xpath", r"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
        #select.select_by_index(0)
        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("table", attrs = {"class": "Crom_table__p1iZz"})
        headers = table.findAll('th')
        headerlist = [h.text.strip() for h in headers[1:]]
        filter_header_list = [a for a in headerlist if not 'RANK' in a]
        rows = table.findAll('tr')[1:]
        player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]
        #filter_header_list = filter_header_list[:-5]
        stats = pd.DataFrame(player_stats, columns = filter_header_list)
        #pd.DataFrame.to_excel(stats, f"/home/rohan/python-projects/jupyter/BasketballProject/miscteam{season}REG.xlsx")
        print(season + " done scraping")
''' 

In [ ]:
#get_data(seasons)